###🚀 We recommend using a runtime with GPU to run this notebook

In [1]:
from encodecmae import load_model
import torch

num_gpus = torch.cuda.device_count()
if num_gpus>0:
  device='cuda:0'
else:
  device='cpu'
model = load_model('ec-ec-base_st', device=device)

In [3]:
import librosa
import numpy as np
from pathlib import Path

batch_size = 8
some_wavs = list(Path('../../gsc/eight').rglob('*.wav'))[:batch_size]

wavs = []
wav_lens = []
for f in some_wavs:
  x, fs = librosa.load(f, sr=model.wav_encoder.fs)
  wavs.append(x)
  wav_lens.append(x.shape[0])
max_wav_len = max(wav_lens)
wavs = np.stack([np.pad(x, (0,max_wav_len - x.shape[0])) for x in wavs])
print(wavs.shape)
# model.visible_encoder.compile=False
# features = model.extract_features_from_array(wavs, layer=-1)

(8, 24000)


In [4]:
model.visible_encoder.compile=False
features = model.extract_features_from_array(wavs, layer=-1)
print("features shape: ", features.shape)

features shape:  (8, 75, 768)


In [5]:
import ruamel.yaml as yaml
import librosa
import random
import numpy as np
config_file_path = './settings/mae.yaml'

with open(config_file_path, "r") as f:
        config = yaml.safe_load(f)

# Load audio signal file
wav_file_path = '../../dac/audio_samples/at2_cvt.wav'
waveform, _ = librosa.load(wav_file_path, sr=config["audio_args"]["sr"], mono=True)
print('waveform shape before crop: ', waveform.shape)
if config["audio_args"]["max_length"] != 0:
            # if audio length is longer than max_length, we random crop it
            max_length = config["audio_args"]["max_length"] * config["audio_args"]["sr"]
            if waveform.shape[-1] > max_length:
                max_start = waveform.shape[-1] - max_length
                start = random.randint(0, max_start)
                waveform = waveform[start: start + max_length]
                
print('waveform shape: ', waveform.shape)
waveform_tensor = torch.tensor(waveform[None, :])
print('waveform_tensor shape: ', waveform_tensor.shape)

waveform shape before crop:  (360001,)
waveform shape:  (240000,)
waveform_tensor shape:  torch.Size([1, 240000])


In [6]:
batch_size = 5
batch_waveform_tensor = waveform_tensor.repeat(batch_size, 1)
print(batch_waveform_tensor.shape)

torch.Size([5, 240000])


In [7]:
model.visible_encoder.compile=False
features = model.extract_features_from_array(batch_waveform_tensor, layer=-1)
print(features.shape)

(5, 750, 768)


In [14]:
features_len = np.array(batch_waveform_tensor)//model.wav_encoder.hop_length
features_len.shape

(5, 240000)

In [13]:
features = model.extract_features_from_array(batch_waveform_tensor.to(model.device, model.dtype), layer=-1, return_type='torch') #And to generate torch tensors outputs
features.shape

torch.Size([5, 750, 768])

In [ ]:
import torch
import torch.nn as nn
from models.audio_encoder import AudioEncoder
from models.text_encoder import TextEncoder
import torch.nn.functional as F
import copy
from tools.losses import AudioTextContrastiveLoss, NTXent
from tools.utils import remove_grad
import ruamel.yaml as yaml
import librosa
import random
import numpy as np

config_file_path = './retrieval/settings/vamp.yaml'

with open(config_file_path, "r") as f:
        config = yaml.safe_load(f)
        
# Load audio signal file
wav_file_path = '../dac/audio_samples/at2_cvt.wav'
waveform, _ = librosa.load(wav_file_path, sr=config["audio_args"]["sr"], mono=True)
print('waveform shape before crop: ', waveform.shape)
if config["audio_args"]["max_length"] != 0:
            # if audio length is longer than max_length, we random crop it
            max_length = config["audio_args"]["max_length"] * config["audio_args"]["sr"]
            if waveform.shape[-1] > max_length:
                max_start = waveform.shape[-1] - max_length
                start = random.randint(0, max_start)
                waveform = waveform[start: start + max_length]
                
print('waveform shape: ', waveform.shape)
waveform_tensor = torch.tensor(waveform[None, :])
print('waveform_tensor shape: ', waveform_tensor.shape)

batch_size = 5
batch_waveform_tensor = waveform_tensor.repeat(batch_size, 1)
print("batch_waveform_tensor.shape: ", batch_waveform_tensor.shape)

audio_encoder = AudioEncoder(config)
# settings for projection layers
embed_size = config["embed_size"]
audio_width = audio_encoder.audio_width

audio_feats = audio_encoder(batch_waveform_tensor)
# audio_embeds = F.normalize(self.audio_proj(audio_feats), dim=-1)
# print(audio_feats)
print('audio_encoded.shape: ', audio_feats.shape)
# audio_feats
print("The similarity of 0-5 is: ", torch.dist(audio_feats[0], audio_feats[4]))
